<a target="_blank" href="https://colab.research.google.com/github/menloparklab/falcon-langchain/blob/main/falcon.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# !pip install langchain huggingface_hub > /dev/null

In [1]:
# Run this if you have added HUGGINGFACEHUB_API_TOKEN as an environment variable
import os
huggingfacehub_api_token = os.environ['HUGGINGFACEHUB_API_TOKEN']


# huggingfacehub_api_token="your_api_token_here"

In [2]:
import os
huggingfacehub_api_token = os.environ['HUGGINGFACEHUB_API_TOKEN']

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.runnable.config import RunnableConfig
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chains.summarize import load_summarize_chain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import chainlit as cl

repo_id = "tiiuae/falcon-7b-instruct"
model = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token, 
                    repo_id=repo_id, 
                    model_kwargs={"temperature":0.1, "max_new_tokens":500}
                    )
vectorstore = FAISS.from_texts(
    ["Reggie Bain is the CEO of Apple in 2023"], embedding=HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
            {context}
            Question: {question}
            """
template2 = 'Answer'
prompt = PromptTemplate.from_template(template)


    
chain = RetrievalQAWithSourcesChain.from_chain_type(
    HuggingFaceHub(
        huggingfacehub_api_token=huggingfacehub_api_token,
        repo_id=repo_id,
        model_kwargs={'temperature':0.1},
    ),
    chain_type="stuff",
    retriever=retriever,
)
# Parser required when using ChatPromptTemplate since it returns ChatPromptValue
output_parser = StrOutputParser()

# Add a tool that is used every time
search = DuckDuckGoSearchRun()
runnable = {'context': retriever, 'question': RunnablePassthrough()} | prompt | model | output_parser
#runnable = prompt | chain | output_parser
runnable.invoke("Who is the CEO of apple in 2023?")

2023-12-20 19:21:15 - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


/home/codespace/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


2023-12-20 19:21:15 - Use pytorch device: cpu


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.45it/s]
/home/codespace/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
Batches: 100%|██████████| 1/1 [00:00<00:00, 17.01it/s]


'<p>Reggie Bain is the CEO of Apple in 2023.</p>'

In [8]:
from langchain import PromptTemplate, LLMChain

template = """
You are an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

{question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
prompt2 = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "How to make pizza?"

print(llm_chain.run(question))

1. Preheat your oven to 475°F (245°C).
2. Roll out your pizza dough and spread it onto a baking sheet.
3. Add your desired tomato sauce and toppings.
4. Bake the pizza in the oven for 15 minutes.
5. Take the pizza out and let it cool for a few minutes.
6. Slice and serve.
User 


In [4]:
print(prompt)

input_variables=['question'] template="\nYou are an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\n\n{question}\n"
